In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 运行 TensorFlow 模型的超参数调优

<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Ftraining%2Fhyperparameter_tuning_tensorflow.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>  
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>                                                                                        
</table>

## 概览

您为模型的超参数选择的数值可能会有很大的差异。如果您只是尝试调整少量超参数，也许可以手动运行实验。但是当您开始操纵模型架构、优化器，以及找到最佳批次大小和学习率的超参数时，将这些实验自动化并进行规模化变得必不可少。

这不仅仅是跟踪所有这些试验的结果。您还希望以一种高效的方式搜索可能数值空间，以便减少试验那些产生低准确度分数的组合所花费的时间。

Vertex AI Training 包括一个超参数调整服务。 Vertex AI 超参数调整作业会运行多次您的训练代码试验。在每次试验中，它会使用不同数值来设置您选择的超参数，且在您指定的限制内。默认情况下，该服务使用贝叶斯优化算法来搜索可能超参数数值空间。这意味着会利用先前实验的信息来选择下一组数值，使搜索更加高效。

了解有关[Vertex AI超参数调整](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)的更多信息。

### 目标

在本教程中，您将学习如何为 TensorFlow 模型运行 Vertex AI 超参数调整作业。虽然此示例使用 TensorFlow，但您也可以将此服务用于其他 ML 框架。

本教程使用以下 Google Cloud ML 服务和资源:

* Vertex AI 训练
* 云存储
* Artifact Registry

执行的步骤包括:

* 修改用于自动化超参数调整的训练应用程序代码。
* 为训练应用程序代码制作容器。
* 使用 Vertex AI Python SDK 配置和启动超参数调整作业。

数据集

这个样本使用通过[TensorFlow数据集](https://www.tensorflow.org/datasets)提供的[马或人数据集](https://www.tensorflow.org/datasets/catalog/horses_or_humans)来训练一个二元图像分类模型（马或人）。

成本

本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- Cloud Storage
- Artifact Registry

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 价格](https://cloud.google.com/storage/pricing) 和 [Artifact Registry 价格](https://cloud.google.com/artifact-registry/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

开始吧

安装Vertex AI SDK for Python和其他所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### 重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续进行下一步之前，请等待它完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置 Google Cloud 项目信息并初始化用于 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解有关[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的更多信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type: "string"}

UUID

如果您正在参加一个实时教程会话，您可能正在使用一个共享的测试账户或项目。为了避免资源创建时用户之间的名称冲突，为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有如果您的存储桶不存在：运行以下内容以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

### 为 Python 初始化 VertexAI SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

容器化培训应用代码

在你能够运行超参数调优工作之前，你必须创建一个源代码文件（训练脚本）和一个Dockerfile。

源代码训练一个模型，使用你选择的ML框架。在这个例子中，你使用TensorFlow来训练一个分类模型。

Dockerfile包括运行镜像所需的所有命令。它安装训练脚本需要的所有库，并设置训练代码的入口点。

你会创建一个目录来存储你的源代码文件和Dockerfile。

In [ ]:
APPLICATION_DIR = "hptune"
TRAINER_DIR = f"{APPLICATION_DIR}/trainer"

In [ ]:
!mkdir -p $APPLICATION_DIR
!mkdir -p $TRAINER_DIR

### 写培训脚本

In [ ]:
%%writefile {TRAINER_DIR}/task.py

import tensorflow as tf
import tensorflow_datasets as tfds
import argparse
import hypertune

NUM_EPOCHS = 10


def get_args():
  '''Parses args. Must include all hyperparameters you want to tune.'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate',
      required=True,
      type=float,
      help='learning rate')
  parser.add_argument(
      '--momentum',
      required=True,
      type=float,
      help='SGD momentum value')
  parser.add_argument(
      '--num_units',
      required=True,
      type=int,
      help='number of units in last hidden layer')
  args = parser.parse_args()
  return args


def preprocess_data(image, label):
  '''Resizes and scales images.'''

  image = tf.image.resize(image, (150,150))
  return tf.cast(image, tf.float32) / 255., label


def create_dataset():
  '''Loads Horses Or Humans dataset and preprocesses data.'''

  data, info = tfds.load(name='horses_or_humans', as_supervised=True, with_info=True)

  # Create train dataset
  train_data = data['train'].map(preprocess_data)
  train_data  = train_data.shuffle(1000)
  train_data  = train_data.batch(64)

  # Create validation dataset
  validation_data = data['test'].map(preprocess_data)
  validation_data  = validation_data.batch(64)

  return train_data, validation_data


def create_model(num_units, learning_rate, momentum):
  '''Defines and compiles model.'''

  inputs = tf.keras.Input(shape=(150, 150, 3))
  x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(inputs, outputs)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum),
      metrics=['accuracy'])
  return model


def main():
  args = get_args()
  train_data, validation_data = create_dataset()
  model = create_model(args.num_units, args.learning_rate, args.momentum)
  history = model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data)

  # DEFINE METRIC
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=NUM_EPOCHS)


if __name__ == "__main__":
    main()

### 理解训练脚本

在构建容器之前，让我们深入了解代码。有一些部分是特定于使用超参数调整服务的。

脚本导入了 **hypertune库**：

`import hypertune`

**函数 `get_args()`** 定义了要调整的每个超参数的命令行参数。在这个例子中，调整的超参数是学习率、优化器中的动量值，以及模型中最后一个隐藏层中的单位数。随意尝试其他参数。传递给这些参数的值然后用来设置代码中对应的超参数。

```
def get_args():
'''解析参数。必须包括您想要调整的所有超参数。'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate',
      required=True,
      type=float,
      help='学习率')
  parser.add_argument(
      '--momentum',
      required=True,
      type=float,
      help='SGD动量值')
  parser.add_argument(
      '--num_units',
      required=True,
      type=int,
      help='最后隐藏层中的单位数')
  args = parser.parse_args()
  return args
```


在 `main()` 函数的末尾，使用 `hypertune` 库来 **定义要优化的指标**。在 TensorFlow 中，keras 的 `model.fit` 方法返回一个 `History` 对象。`History.history` 属性是在连续的epochs中训练损失值和指标值的记录。如果将验证数据传递给 `model.fit`，则 `History.history` 属性还包括验证损失和指标值。

```
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=NUM_EPOCHS)
 ```
 
例如，如果您对具有验证数据的模型进行了三次epochs的训练，并提供了准确性作为指标，`History.history` 属性看起来类似于下面的字典。

```
{
 "accuracy": [
   0.7795261740684509,
   0.9471358060836792,
   0.9870933294296265
 ],
 "loss": [
   0.6340447664260864,
   0.16712145507335663,
   0.04546636343002319
 ],
 "val_accuracy": [
   0.3795261740684509,
   0.4471358060836792,
   0.4870933294296265
 ],
 "val_loss": [
   2.044623374938965,
   4.100203514099121,
   3.0728273391723633
 ]
```

如果您希望超参数调整服务发现最大化模型的验证准确性的值，可以将指标定义为 `val_accuracy` 列表的最后一个条目（或 `NUM_EPOCHS` - 1）。然后，将此指标传递给 `HyperTune` 的一个实例。您可以选择任意字符串作为 `hyperparameter_metric_tag`，但您需要在稍后启动超参数调整作业时再次使用该字符串。

### 编写 Dockerfile

在编写完您的训练代码后，您需要创建一个 Dockerfile。在 Dockerfile 中，包括运行镜像所需的所有命令。它安装了所有必要的库，包括 CloudML Hypertune 库，并设置了训练代码的入口点。

In [ ]:
%%writefile {APPLICATION_DIR}/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8

WORKDIR /

# Installs hypertune library
RUN pip install cloudml-hypertune

# Copies the trainer code to the Docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

### 启用Artifact Registry API

首先，您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

### 构建容器

您将Docker镜像存储在Artifact Registry中。首先，在Artifact Registry中创建一个Docker存储库。

In [ ]:
import os

REPO_NAME='horses-app'

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

!gcloud artifacts repositories create $REPO_NAME --repository-format=docker \
--location=$LOCATION --description="Docker repository"

### 配置私有仓库的身份验证

在推送或拉取容器镜像之前，请配置Docker使用 `gcloud` 命令行工具来对您所在地区的 `Artifact Registry` 进行请求身份验证。

In [ ]:
! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet

在Artifact Registry中定义一个变量，其值为您Docker镜像的URI。

In [ ]:
IMAGE_URI = (
    f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/horse_human_hptune:latest"
)

然后，通过运行以下命令构建容器并将其推送到Artifact Registry:

In [ ]:
cd $APPLICATION_DIR

使用Cloud-Build将图像推送到创建的存储库中。

**注意**：以下命令会自动使用当前目录中的Dockerfile。

In [ ]:
! gcloud builds submit --region={LOCATION} --tag={IMAGE_URI}

## 配置超参数调整作业

现在您的训练应用程序代码已经容器化，是时候指定并运行超参数调整作业了。

要启动超参数调整作业，您需要首先定义`worker_pool_specs`，指定机器类型和Docker镜像。以下规范定义了一个`n1-standard-4`机器，配备一个`NVIDIA Tesla T4` GPU作为加速器。

In [ ]:
# The spec of the worker pools including machine type and Docker image
# Be sure to replace PROJECT_ID in the `image_uri` with your project.

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/horse_human_hptune:latest"
        },
    }
]

### 定义参数规范

接下来，定义 `parameter_spec`，它是一个字典，用于指定您想要优化的参数。**字典键**是您为每个超参数分配的命令行参数字符串，而**字典值**是参数规范。

对于每个超参数，您需要定义`Type`以及调整服务尝试的值的边界。超参数可以是`Double`、`Integer`、`Categorical`或`Discrete`类型。如果选择`Double`或`Integer`类型，则需要提供最小和最大值。如果选择`Categorical`或`Discrete`类型，则需要提供值。对于`Double`和`Integer`类型，您还需要提供缩放值。了解更多关于[使用合适的缩放](https://www.youtube.com/watch?v=cSoK_6Rkbfg)。

In [ ]:
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "num_units": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None),
}

要定义的最终规范是`metric_spec`，它是表示要优化的度量的字典。字典键是您在培训应用程序代码中设置的`hyperparameter_metric_tag`，值是优化目标。

In [ ]:
# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec = {"accuracy": "maximize"}

一旦规格定义完成，您可以创建一个`CustomJob`，这是用于在每个超参数调优试验中运行作业的常见规格。

In [ ]:
my_custom_job = aiplatform.CustomJob(
    display_name="horses-humans-sdk-job",
    worker_pool_specs=worker_pool_specs,
    staging_bucket=BUCKET_URI,
)

然后，创建并运行`HyperparameterTuningJob`。

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name="horses-humans-sdk-job",
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=10,
    parallel_trial_count=3,
)

hp_job.run()

有几点需要注意的论点：

* **max_trial_count**：您需要对服务运行的试验次数设置一个上限。更多的试验通常会带来更好的结果，但存在着收益递减的点，在此之后，额外的试验对您试图优化的指标几乎没有或没有任何影响。最佳做法是从较少的试验开始，然后了解您选择的超参数的影响力，再逐渐扩大试验次数。

* **parallel_trial_count**：如果使用并行试验，服务会为多个训练处理集群配置资源。增加并行试验的数量会减少超参数调整作业运行所需的时间；但是，这可能会降低作业的整体效果。这是因为默认的调整策略使用前一次试验的结果来指导后续试验中参数值的分配。

* **search_algorithm**：您可以将搜索算法设置为grid、random或default（None）。如果您没有指定算法，如本示例所示，系统会应用贝叶斯优化算法搜索可能的超参数值空间，这是推荐的算法。了解更多关于使用贝叶斯优化进行超参数调整的信息：[使用贝叶斯优化进行超参数调整](https://cloud.google.com/blog/products/ai-machine-learning/hyperparameter-tuning-cloud-machine-learning-engine-using-bayesian-optimization)。

检查结果

点击输出中生成的链接在云控制台中查看您的运行情况。当作业完成时，您将看到调整试验的结果。

您可以按照优化指标对结果进行排序，然后将超参数设置在您的训练应用程序代码中，以将值设定为具有最高准确性的试验结果。

将以下英文文本翻译为中文：![console_ui_results](tuning_results.png)

清理

要清理此项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的单个资源：

In [ ]:
# Delete artifact registry repo
! gcloud artifacts repositories delete $REPO_NAME --location $LOCATION --quiet

delete_custom_job = True
delete_application_directory = True
delete_bucket = False

# Delete hptune job
if delete_custom_job:
    try:
        hp_job.delete()
    except Exception as e:
        print(e)

# Delete application directory
if delete_application_directory:
    !cd .. && rm -rf $APPLICATION_DIR

# Delete bucket
if delete_bucket:
    ! gsutil rm -r $BUCKET_URI